<a href="https://colab.research.google.com/github/dulhara79/k_dramalist-data-extractor/blob/main/k_dramalist_data_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 20.1 MB/s eta 0:00:00
171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 45.7s171.6 MiB [] 0% 34.3s171.6 MiB [] 0% 15.9s171.6 MiB [] 0% 8.6s171.6 MiB [] 1% 6.7s171.6 MiB [] 1% 5.8s171.6 MiB [] 2% 5.3s171.6 MiB [] 2% 4.5s171.6 MiB [] 3% 4.1s171.6 MiB [] 4% 3.9s171.6 MiB [] 4% 3.7s171.6 MiB [] 5% 3.6s171.6 MiB [] 5% 3.8s171.6 MiB [] 6% 3.8s171.6 MiB [] 6% 3.7s171.6 MiB [] 7% 3.6s171.6 MiB [] 8% 3.4s171.6 MiB [] 9% 3.4s171.6 MiB [] 10% 3.3s171.6 MiB [] 11% 3.1s171.6 MiB [] 12% 3.0s171.6 MiB [] 13% 2.9s171.6 MiB [] 14% 2.9s171.6 MiB [] 15% 2.8s171.6 MiB [] 16% 2.7s171.6 MiB [] 17% 2.6s171.6 MiB [] 18% 2.5s171.6 MiB [] 19% 2.5s171.6 MiB [] 19% 2.4s171.6 MiB [] 20% 2.4s171.6 MiB [] 21% 2.4s171.6 MiB [] 22% 2.4s171.6 MiB [] 23% 2.3s171.6 MiB [] 24% 2.2s171.6 MiB [] 25% 2.2s171.6 MiB [] 26% 2.1s171.6 MiB [] 26% 2.2s171.6 MiB [] 27% 2.2s171.6 MiB [] 28% 2.1s171.6 MiB [] 29% 2.0s171.6 MiB [] 31% 2.0s171.6 MiB [] 32% 1.9s171.6 MiB [] 33% 1.8s171.6 MiB [] 

In [2]:
from playwright.async_api import async_playwright
import asyncio
from bs4 import BeautifulSoup
import re
import csv
import unicodedata
import math
import logging

In [3]:
# Setup logging
logging.basicConfig(filename="scraper.log", level=logging.INFO, format="%(asctime)s - %(message)s")

In [4]:
def clean_text(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if ord(c) < 128)

In [5]:
def safe_get_text(element, selector, default=""):
    found = element.select_one(selector) if element else None
    return clean_text(found.get_text(strip=True)) if found else default

In [6]:
def safe_get_list(element, selector, default=[]):
    found = element.select(selector) if element else None
    return [clean_text(item.get_text(strip=True)) for item in found] if found else default

In [7]:
async def goto_with_retry(page, url, retries=3, timeout=120000):
    for attempt in range(retries):
        try:
            await page.goto(url, timeout=timeout, wait_until='domcontentloaded')
            return
        except Exception as e:
            if attempt < retries - 1:
                print(f"Retrying {url}... attempt {attempt + 1}")
                await asyncio.sleep(5)
            else:
                raise e

In [8]:
async def get_drama_links(page_num):
    url = f"https://mydramalist.com/search?adv=titles&ty=68&co=3&st=3&page={page_num}"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0")
        page = await context.new_page()
        await goto_with_retry(page, url)
        await page.wait_for_selector("h6.title a", timeout=60000)
        content = await page.content()
        await browser.close()

        soup = BeautifulSoup(content, 'html.parser')
        links = []
        for title in soup.select("h6.title a[href^='/']"):
            href = title.get("href")
            if not href.startswith('/search'):
                links.append("https://mydramalist.com" + href)
        return links

In [9]:
# async def get_drama_links(page_num):
#     url = f"https://mydramalist.com/search?adv=titles&ty=68&co=3&st=3&page={page_num}"
#     async with async_playwright() as p:
#         browser = await p.chromium.launch(headless=True)
#         context = await browser.new_context(user_agent="Mozilla/5.0")
#         page = await context.new_page()
#         await page.goto(url, timeout=60000, wait_until='domcontentloaded')
#         await page.wait_for_selector("h6.title a", timeout=30000) # Increased timeout
#         content = await page.content()
#         soup = BeautifulSoup(content, 'html.parser')
#         await browser.close()

#         # Extract drama links
#         links = []
#         for title in soup.select("h6.title a[href^='/']"):
#             href = title.get("href")
#             if not href.startswith('/search'):
#                 links.append("https://mydramalist.com" + href)
#         return links

In [10]:
# async def get_drama_details(url):
#     try:
#         async with async_playwright() as p:
#             browser = await p.chromium.launch(headless=True)
#             context = await browser.new_context(user_agent="Mozilla/5.0")
#             page = await context.new_page()
#             await page.goto(url, timeout=60000, wait_until='domcontentloaded')
#             content = await page.content()
#             soup = BeautifulSoup(content, 'html.parser')
#             await browser.close()

#             def safe_get_text(element, selector, default=""):
#                 found = element.select_one(selector)
#                 return found.get_text(strip=True) if found else default

#             # Extract title
#             title = safe_get_text(soup, "h1.film-title")

#             details_box = soup.select_one("div.box-body.light-b")

#             # Extract year
#             year_element = soup.select_one("span.release-year") or soup.select_one("div.film-subtitle")
#             year_text = year_element.get_text(strip=True) if year_element else ""

#             # Extract the year using regular expression
#             year_match = re.search(r'(\d{4})(?!.*\d)', year_text)  # Finds last 4-digit number
#             year = year_match.group(1) if year_match else ""

#             # Extract aired date
#             aired_text = safe_get_text(details_box, "li.list-item:has(b:contains('Aired'))")
#             aired_match = re.search(r'(\w+\s+\d{1,2},\s+\d{4})\s*-\s*(\w+\s+\d{1,2},\s+\d{4})?', aired_text)
#             start_date = aired_match.group(1) if aired_match else ""
#             end_date = aired_match.group(2) if aired_match and aired_match.group(2) else ""

#             # Extract original network
#             network = safe_get_text(details_box, "li.list-item:has(b:contains('Original Network'))").replace("Original Network:", "").strip()

#             # Extract aired on
#             aired_on = safe_get_text(details_box, "li.list-item:has(b:contains('Aired On'))").replace("Aired On:", "").strip()

#             # Extract number of episodes
#             episodes = safe_get_text(details_box, "li.list-item:has(b:contains('Episodes'))").replace("Episodes:", "").strip()

#             # Extract duration
#             duration = safe_get_text(details_box, "li.list-item:has(b:contains('Duration'))").replace("Duration:", "").strip()

#             # Extract content rating
#             content_rating = safe_get_text(details_box, "li.list-item:has(b:contains('Content Rating'))").replace("Content Rating:", "").strip()

#             # Extract rating
#             rating = safe_get_text(soup, "div.hfs > b")

#             # Extract rating (score)
#             score = safe_get_text(details_box, "li.list-item:has(b:contains('Score'))").replace("Score:", "").strip()

#             # Extract genres
#             genres = [g.get_text(strip=True) for g in soup.select("li.list-item a[href*='/search?adv=titles&ge=']")] if soup.select("li.list-item a[href*='/search?adv=titles&ge=']") else []

#             # Extract tags
#             tags = [t.get_text(strip=True) for t in soup.select("li.show-tags a[href*='/search?adv=titles&th=']")] if soup.select("li.show-tags a[href*='/search?adv=titles&th=']") else []

#             # Extract synopsis
#             synopsis = safe_get_text(soup, "div.show-synopsis")

#             # Extract statistics
#             ranked = safe_get_text(details_box, "li.list-item:has(b:contains('Ranked'))").replace("Ranked:", "").strip()
#             popularity = safe_get_text(details_box, "li.list-item:has(b:contains('Popularity'))").replace("Popularity:", "").strip()
#             watchers = safe_get_text(details_box, "li.list-item:has(b:contains('Watchers'))").replace("Watchers:", "").strip()

#             # Extract directors
#             directors = [d.get_text(strip=True) for d in soup.select("li.list-item:has(b:contains('Director')) a.text-primary")] if soup.select("li.list-item:has(b:contains('Director')) a.text-primary") else []

#             # Extract screenwriters
#             screenwriters = [w.get_text(strip=True) for w in soup.select("li.list-item:has(b:contains('Screenwriter')) a.text-primary")] if soup.select("li.list-item:has(b:contains('Screenwriter')) a.text-primary") else []

#             # Extract main actors and their character names from Cast & Crew section
#             main_actors = []
#             cast_elements = soup.select("ul.credits li.list-item")[:4] if soup.select("ul.credits li.list-item") else []
#             for cast in cast_elements:
#                 actor_name = safe_get_text(cast, "b[itemprop='name']")
#                 character_element = cast.select_one("small a.text-primary") or cast.select_one("small")
#                 character_name = character_element.get_text(strip=True) if character_element else ""
#                 role_type = "Main Role" if "Main Role" in cast.get_text(strip=True) else "Supporting Role"
#                 main_actors.append({
#                     "name": actor_name,
#                     "character": character_name,
#                     "role_type": role_type
#                 })

#             return {
#                 "title": title,
#                 "year": year,
#                 "start_date": start_date,
#                 "end_date": end_date,
#                 "original_network": network,
#                 "aired_on": aired_on,
#                 "number_of_episodes": episodes,
#                 "duration": duration,
#                 "content_rating": content_rating,
#                 "rating": rating,
#                 "genres": genres,
#                 "tags": tags,
#                 "synopsis": synopsis,
#                 "statistics": {
#                     "score": score,
#                     "ranked": ranked,
#                     "popularity": popularity,
#                     "watchers": watchers
#                 },
#                 "directors": directors,
#                 "screenwriters": screenwriters,
#                 "main_actors": main_actors,
#                 "url": url
#             }
#     except Exception as e:
#         print(f"Failed to fetch {url}: {str(e)}")
#         return None


In [11]:
# async def get_drama_details(url):
#     try:
#         async with async_playwright() as p:
#             browser = await p.chromium.launch(headless=True)
#             context = await browser.new_context(user_agent="Mozilla/5.0")
#             page = await context.new_page()
#             await page.goto(url, timeout=60000, wait_until='domcontentloaded')
#             content = await page.content()
#             soup = BeautifulSoup(content, 'html.parser')
#             await browser.close()

#             def safe_get_text(element, selector, default=""):
#                 found = element.select_one(selector)
#                 return found.get_text(strip=True) if found else default

#             # Extract fields with safe handling
#             title = safe_get_text(soup, "h1.film-title")

#             # Extract year from film-subtitle if release-year not found
#             year_element = soup.select_one("span.release-year") or soup.select_one("div.film-subtitle")
#             year_text = year_element.get_text(strip=True) if year_element else ""

#             # Extract the year using regular expression
#             year_match = re.search(r'(\d{4})(?!.*\d)', year_text)  # Finds last 4-digit number
#             year = year_match.group(1) if year_match else ""

#             rating = safe_get_text(soup, "div.hfs > b")

#             # Extract genres
#             genres = [g.get_text(strip=True) for g in soup.select("li.list-item a[href*='/search?adv=titles&ge=']")] if soup.select("li.list-item a[href*='/search?adv=titles&ge=']") else []

#             # Extract tags
#             tags = [t.get_text(strip=True) for t in soup.select("li.show-tags a[href*='/search?adv=titles&th=']")] if soup.select("li.show-tags a[href*='/search?adv=titles&th=']") else []

#             # Extract synopsis
#             synopsis = safe_get_text(soup, "div.show-synopsis")

#             # Extract statistics
#             stats_box = soup.select_one("div.box-body.light-b")
#             score = safe_get_text(stats_box, "li.list-item:has(b:contains('Score'))", "").replace("Score", "").strip()
#             ranked = safe_get_text(stats_box, "li.list-item:has(b:contains('Ranked'))", "").replace("Ranked", "").strip()
#             popularity = safe_get_text(stats_box, "li.list-item:has(b:contains('Popularity'))", "").replace("Popularity", "").strip()
#             watchers = safe_get_text(stats_box, "li.list-item:has(b:contains('Watchers'))", "").replace("Watchers", "").strip()

#             # Extract directors
#             directors = [d.get_text(strip=True) for d in soup.select("li.list-item:has(b:contains('Director')) a.text-primary")] if soup.select("li.list-item:has(b:contains('Director')) a.text-primary") else []

#             # Extract screenwriters
#             screenwriters = [w.get_text(strip=True) for w in soup.select("li.list-item:has(b:contains('Screenwriter')) a.text-primary")] if soup.select("li.list-item:has(b:contains('Screenwriter')) a.text-primary") else []

#             # Extract main actors (top 4 listed in cast section)
#             main_actors = []
#             actor_elements = soup.select("ul.credits li.list-item")[:4] if soup.select("ul.credits li.list-item") else []
#             for actor in actor_elements:
#                 actor_name = safe_get_text(actor, "b[itemprop='name']")
#                 character_element = actor.select_one("small a.text-primary") or actor.select_one("small")
#                 character_name = character_element.get_text(strip=True) if character_element else ""
#                 main_actors.append({
#                     "name": actor_name,
#                     "character": character_name
#                 })

#             return {
#                 "title": title,
#                 "year": year,
#                 "rating": rating,
#                 "genres": genres,
#                 "tags": tags,
#                 "synopsis": synopsis,
#                 "statistics": {
#                     "score": score,
#                     "ranked": ranked,
#                     "popularity": popularity,
#                     "watchers": watchers
#                 },
#                 "directors": directors,
#                 "screenwriters": screenwriters,
#                 "main_actors": main_actors,
#                 "url": url
#             }
#     except Exception as e:
#         print(f"Failed to fetch {url}: {str(e)}")
#         return None

In [12]:
# async def get_drama_details(url):
#     try:
#         async with async_playwright() as p:
#             browser = await p.chromium.launch(headless=True)
#             context = await browser.new_context(user_agent="Mozilla/5.0")
#             page = await context.new_page()
#             await page.goto(url, timeout=60000, wait_until='domcontentloaded')
#             content = await page.content()
#             soup = BeautifulSoup(content, 'html.parser')
#             await browser.close()

#             def safe_get_text(element, selector, default=""):
#                 found = element.select_one(selector)
#                 return clean_text(found.get_text(strip=True)) if found else default

#             def safe_get_list(element, selector, default=[]):
#                 found = element.select(selector)
#                 return [clean_text(item.get_text(strip=True)) for item in found] if found else default

#             def clean_text(text):
#                 # Normalize and keep only ASCII characters
#                 return ''.join(c for c in unicodedata.normalize('NFKD', text) if ord(c) < 128)

#             # Extract title
#             title = safe_get_text(soup, "h1.film-title")

#             # Extract year from film-subtitle
#             year_element = soup.select_one("span.release-year") or soup.select_one("div.film-subtitle")
#             year_text = clean_text(year_element.get_text(strip=True)) if year_element else ""
#             year_match = re.search(r'(\d{4})(?!.*\d)', year_text)
#             year = year_match.group(1) if year_match else ""

#             # Extract rating, set to NaN if not found
#             rating_text = safe_get_text(soup, "div.hfs > b")
#             rating = float(rating_text) if rating_text and rating_text.replace(".", "").isdigit() else float('nan')

#             # Extract genres
#             genres = safe_get_list(soup, "li.show-genres a[href*='/search?adv=titles&ge=']")

#             # Extract tags
#             tags = safe_get_list(soup, "li.show-tags a[href*='/search?adv=titles&th=']")

#             # Extract synopsis
#             synopsis = safe_get_text(soup, "div.show-synopsis p")

#             # Extract statistics
#             stats_box = soup.select_one("div.box-body.light-b")
#             score = safe_get_text(stats_box, "li.list-item:has(b:contains('Score'))", "").replace("Score", "").strip()
#             ranked = safe_get_text(stats_box, "li.list-item:has(b:contains('Ranked'))", "").replace("Ranked", "").strip()
#             popularity = safe_get_text(stats_box, "li.list-item:has(b:contains('Popularity'))", "").replace("Popularity", "").strip()
#             watchers = safe_get_text(stats_box, "li.list-item:has(b:contains('Watchers'))", "").replace("Watchers", "").strip()

#             # Extract directors
#             directors = safe_get_list(soup, "li.list-item:has(b:contains('Director')) a.text-primary")

#             # Extract screenwriters
#             screenwriters = safe_get_list(soup, "li.list-item:has(b:contains('Screenwriter')) a.text-primary")

#             # Extract main actors (all listed in cast section with role details)
#             main_actors = []
#             actor_elements = soup.select("ul.credits li.list-item")
#             for actor in actor_elements:
#                 actor_name = safe_get_text(actor, "b[itemprop='name']")
#                 character_element = actor.select_one("small a.text-primary") or actor.select_one("small")
#                 character_name = clean_text(character_element.get_text(strip=True)) if character_element else ""
#                 role = clean_text(safe_get_text(actor, "small.text-muted").replace("Main Role", "Main").replace("Support Role", "Support"))
#                 main_actors.append({
#                     "name": actor_name,
#                     "character": character_name,
#                     "role": role
#                 })

#             # Extract episode details
#             episodes = safe_get_text(stats_box, "li.list-item:has(b:contains('Episodes'))", "").replace("Episodes", "").strip()

#             # Extract duration
#             duration = safe_get_text(stats_box, "li.list-item:has(b:contains('Duration'))", "").replace("Duration", "").strip()

#             # Extract content rating
#             content_rating = safe_get_text(stats_box, "li.list-item:has(b:contains('Content Rating'))", "").replace("Content Rating", "").strip()

#             # Extract original network
#             original_network = safe_get_list(soup, "li.list-item:has(b:contains('Original Network')) a.text-primary")

#             # Extract airing details
#             aired = safe_get_text(stats_box, "li.list-item:has(b:contains('Aired'))", "").replace("Aired", "").strip()
#             aired_on = safe_get_text(stats_box, "li.list-item:has(b:contains('Aired On'))", "").replace("Aired On", "").strip()

#             # Extract native title
#             native_title = safe_get_text(soup, "li.list-item:has(b:contains('Native Title')) a")

#             # Extract alternative titles
#             alternative_titles = safe_get_list(soup, "li.list-item:has(b:contains('Also Known As')) span.mdl-aka-titles a") or \
#                                [clean_text(t.strip()) for t in safe_get_text(soup, "li.list-item:has(b:contains('Also Known As')) span.mdl-aka-titles").split(",")]

#             # Extract where to watch
#             where_to_watch = []
#             watch_elements = soup.select("div.box-body.wts div.row div.col-xs-12")
#             for watch in watch_elements:
#                 platform = safe_get_text(watch, "a.text-primary b")
#                 access_type = safe_get_text(watch, "div:not(:first-child)")
#                 where_to_watch.append({
#                     "platform": platform,
#                     "access_type": access_type
#                 })

#             # Extract reviews
#             reviews = []
#             review_elements = soup.select("div.review")
#             for review in review_elements:
#                 reviewer = safe_get_text(review, "b a.text-primary")
#                 review_date = safe_get_text(review, "small.datetime")
#                 overall_score = safe_get_text(review, "div.rating-overall span.score")
#                 overall_score = float(overall_score) if overall_score and overall_score.replace(".", "").isdigit() else float('nan')
#                 story_score = safe_get_text(review, "div.list-group.review-rating div:contains('Story') span")
#                 story_score = float(story_score) if story_score and story_score.replace(".", "").isdigit() else float('nan')
#                 acting_score = safe_get_text(review, "div.list-group.review-rating div:contains('Acting/Cast') span")
#                 acting_score = float(acting_score) if acting_score and acting_score.replace(".", "").isdigit() else float('nan')
#                 music_score = safe_get_text(review, "div.list-group.review-rating div:contains('Music') span")
#                 music_score = float(music_score) if music_score and music_score.replace(".", "").isdigit() else float('nan')
#                 rewatch_score = safe_get_text(review, "div.list-group.review-rating div:contains('Rewatch Value') span")
#                 rewatch_score = float(rewatch_score) if rewatch_score and rewatch_score.replace(".", "").isdigit() else float('nan')
#                 review_text = clean_text(safe_get_text(review, "div.review-body"))
#                 helpful_count = safe_get_text(review, "div.user-stats small b").replace("people found this review helpful", "").strip()
#                 helpful_count = int(helpful_count) if helpful_count and helpful_count.isdigit() else 0
#                 reviews.append({
#                     "reviewer": reviewer,
#                     "date": review_date,
#                     "scores": {
#                         "overall": overall_score,
#                         "story": story_score,
#                         "acting": acting_score,
#                         "music": music_score,
#                         "rewatch": rewatch_score
#                     },
#                     "text": review_text,
#                     "helpful_count": helpful_count
#                 })

#             # Extract recommendations
#             recommendations = []
#             rec_elements = soup.select("div.details-recommendations div.rec-item a")
#             for rec in rec_elements:
#                 rec_title = clean_text(rec.get("title", ""))
#                 rec_url = "https://mydramalist.com" + rec.get("href", "") if rec.get("href") else ""
#                 recommendations.append({
#                     "title": rec_title,
#                     "url": rec_url
#                 })

#             # Extract top contributors
#             contributors = []
#             contributor_elements = soup.select("div.title-contributors div.contributor")
#             for contrib in contributor_elements:
#                 username = safe_get_text(contrib, "a.text-primary b")
#                 edits = safe_get_text(contrib, "div div:nth-child(2)").replace("edits", "").strip()
#                 contributors.append({
#                     "username": username,
#                     "edits": edits
#                 })

#             return {
#                 "title": title,
#                 "native_title": native_title,
#                 "alternative_titles": alternative_titles,
#                 "year": year,
#                 "rating": rating,
#                 "genres": genres,
#                 "tags": tags,
#                 "synopsis": synopsis,
#                 "statistics": {
#                     "score": score,
#                     "ranked": ranked,
#                     "popularity": popularity,
#                     "watchers": watchers
#                 },
#                 "directors": directors,
#                 "screenwriters": screenwriters,
#                 "main_actors": main_actors,
#                 "episodes": episodes,
#                 "duration": duration,
#                 "content_rating": content_rating,
#                 "original_network": original_network,
#                 "aired": aired,
#                 "aired_on": aired_on,
#                 "where_to_watch": where_to_watch,
#                 "reviews": reviews,
#                 "recommendations": recommendations,
#                 "contributors": contributors,
#                 "url": url
#             }
#     except Exception as e:
#         print(f"Failed to fetch {url}: {str(e)}")
#         return None

In [13]:
async def get_drama_details(url):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context(user_agent="Mozilla/5.0")
            page = await context.new_page()
            await goto_with_retry(page, url)
            content = await page.content()
            await browser.close()

            soup = BeautifulSoup(content, 'html.parser')
            stats_box = soup.select_one("div.box-body.light-b")
            if stats_box is None:
                print(f"Missing stats box for {url}")
                return None

            title = safe_get_text(soup, "h1.film-title")
            year_element = soup.select_one("span.release-year") or soup.select_one("div.film-subtitle")
            year_text = clean_text(year_element.get_text(strip=True)) if year_element else ""
            year_match = re.search(r'(\d{4})(?!.*\d)', year_text)
            year = year_match.group(1) if year_match else ""

            rating_text = safe_get_text(soup, "div.hfs > b")
            rating = float(rating_text) if rating_text and rating_text.replace(".", "").isdigit() else float('nan')
            genres = safe_get_list(soup, "li.show-genres a[href*='/search?adv=titles&ge=']")
            tags = safe_get_list(soup, "li.show-tags a[href*='/search?adv=titles&th=']")
            synopsis = safe_get_text(soup, "div.show-synopsis p")

            score = safe_get_text(stats_box, "li.list-item:has(b:contains('Score'))", "").replace("Score", "").strip()
            ranked = safe_get_text(stats_box, "li.list-item:has(b:contains('Ranked'))", "").replace("Ranked", "").strip()
            popularity = safe_get_text(stats_box, "li.list-item:has(b:contains('Popularity'))", "").replace("Popularity", "").strip()
            watchers = safe_get_text(stats_box, "li.list-item:has(b:contains('Watchers'))", "").replace("Watchers", "").strip()

            directors = safe_get_list(soup, "li.list-item:has(b:contains('Director')) a.text-primary")
            screenwriters = safe_get_list(soup, "li.list-item:has(b:contains('Screenwriter')) a.text-primary")

            actor_elements = soup.select("ul.credits li.list-item")
            main_actors = []
            for actor in actor_elements:
                actor_name = safe_get_text(actor, "b[itemprop='name']")
                character_element = actor.select_one("small a.text-primary") or actor.select_one("small")
                character_name = clean_text(character_element.get_text(strip=True)) if character_element else ""
                role = clean_text(safe_get_text(actor, "small.text-muted").replace("Main Role", "Main").replace("Support Role", "Support"))
                main_actors.append({
                    "name": actor_name,
                    "character": character_name,
                    "role": role
                })

            episodes = safe_get_text(stats_box, "li.list-item:has(b:contains('Episodes'))", "").replace("Episodes", "").strip()
            duration = safe_get_text(stats_box, "li.list-item:has(b:contains('Duration'))", "").replace("Duration", "").strip()
            content_rating = safe_get_text(stats_box, "li.list-item:has(b:contains('Content Rating'))", "").replace("Content Rating", "").strip()
            original_network = safe_get_list(soup, "li.list-item:has(b:contains('Original Network')) a.text-primary")
            aired = safe_get_text(stats_box, "li.list-item:has(b:contains('Aired'))", "").replace("Aired", "").strip()
            aired_on = safe_get_text(stats_box, "li.list-item:has(b:contains('Aired On'))", "").replace("Aired On", "").strip()

            native_title = safe_get_text(soup, "li.list-item:has(b:contains('Native Title')) a")
            alternative_titles = safe_get_list(soup, "li.list-item:has(b:contains('Also Known As')) span.mdl-aka-titles a")
            if not alternative_titles:
                raw = safe_get_text(soup, "li.list-item:has(b:contains('Also Known As')) span.mdl-aka-titles")
                alternative_titles = [clean_text(t.strip()) for t in raw.split(",")]

            where_to_watch = []
            for watch in soup.select("div.box-body.wts div.row div.col-xs-12"):
                platform = safe_get_text(watch, "a.text-primary b")
                access_type = safe_get_text(watch, "div:not(:first-child)")
                where_to_watch.append({"platform": platform, "access_type": access_type})

            reviews = []
            for review in soup.select("div.review"):
                reviewer = safe_get_text(review, "b a.text-primary")
                review_date = safe_get_text(review, "small.datetime")
                overall_score = safe_get_text(review, "div.rating-overall span.score")
                story_score = safe_get_text(review, "div.list-group.review-rating div:contains('Story') span")
                acting_score = safe_get_text(review, "div.list-group.review-rating div:contains('Acting/Cast') span")
                music_score = safe_get_text(review, "div.list-group.review-rating div:contains('Music') span")
                rewatch_score = safe_get_text(review, "div.list-group.review-rating div:contains('Rewatch Value') span")
                review_text = clean_text(safe_get_text(review, "div.review-body"))
                helpful_count = safe_get_text(review, "div.user-stats small b").replace("people found this review helpful", "").strip()
                reviews.append({
                    "reviewer": reviewer,
                    "date": review_date,
                    "scores": {
                        "overall": float(overall_score) if overall_score.replace(".", "").isdigit() else float('nan'),
                        "story": float(story_score) if story_score.replace(".", "").isdigit() else float('nan'),
                        "acting": float(acting_score) if acting_score.replace(".", "").isdigit() else float('nan'),
                        "music": float(music_score) if music_score.replace(".", "").isdigit() else float('nan'),
                        "rewatch": float(rewatch_score) if rewatch_score.replace(".", "").isdigit() else float('nan')
                    },
                    "text": review_text,
                    "helpful_count": int(helpful_count) if helpful_count.isdigit() else 0
                })

            recommendations = []
            for rec in soup.select("div.details-recommendations div.rec-item a"):
                rec_title = clean_text(rec.get("title", ""))
                rec_url = "https://mydramalist.com" + rec.get("href", "") if rec.get("href") else ""
                recommendations.append({"title": rec_title, "url": rec_url})

            contributors = []
            for contrib in soup.select("div.title-contributors div.contributor"):
                username = safe_get_text(contrib, "a.text-primary b")
                edits = safe_get_text(contrib, "div div:nth-child(2)").replace("edits", "").strip()
                contributors.append({"username": username, "edits": edits})

            return {
                "title": title,
                "native_title": native_title,
                "alternative_titles": alternative_titles,
                "year": year,
                "rating": rating,
                "genres": genres,
                "tags": tags,
                "synopsis": synopsis,
                "statistics": {
                    "score": score,
                    "ranked": ranked,
                    "popularity": popularity,
                    "watchers": watchers
                },
                "directors": directors,
                "screenwriters": screenwriters,
                "main_actors": main_actors,
                "episodes": episodes,
                "duration": duration,
                "content_rating": content_rating,
                "original_network": original_network,
                "aired": aired,
                "aired_on": aired_on,
                "where_to_watch": where_to_watch,
                "reviews": reviews,
                "recommendations": recommendations,
                "contributors": contributors,
                "url": url
            }
    except Exception as e:
        logging.error(f"Failed to fetch {url}: {str(e)}")
        return None

In [14]:
# async def main():
#     all_data = []

#     for page_num in range(1, 103):  # Scrape first 1 pages
#         print(f"Scraping page {page_num}...")
#         url = f"https://mydramalist.com/search?adv=titles&ty=68&co=3&so=rated&page={page_num}"
#         drama_links = await get_drama_links(url)

#         for link in drama_links:
#             data = await get_drama_details(link)
#             if data:  # Only append if data was successfully scraped
#                 all_data.append(data)

#     # Only save to CSV if we have data
#     if all_data:
#         keys = all_data[0].keys()
#         with open('drama_dataset.csv', 'w', newline='', encoding='utf-8') as f:
#             dict_writer = csv.DictWriter(f, keys)
#             dict_writer.writeheader()
#             dict_writer.writerows(all_data)
#         print("Data saved to drama_dataset.csv")
#     else:
#         print("No data was scraped successfully")

In [15]:
async def main():
    all_data = []
    for page_num in range(1, 103):  # Try with 5 pages first
        print(f"Scraping page {page_num}...")
        logging.info(f"Scraping page {page_num}")
        links = await get_drama_links(page_num)
        for link in links:
            print(f"Scraping drama: {link}")
            data = await get_drama_details(link)
            if data:
                all_data.append(data)

    if all_data:
        keys = all_data[0].keys()
        with open('drama_dataset.csv', 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, keys)
            dict_writer.writeheader()
            dict_writer.writerows(all_data)
        print("✅ Data saved to drama_dataset.csv")
    else:
        print("❌ No data scraped")

In [16]:
# # For Jupyter or Colab
# import nest_asyncio
# import asyncio

# nest_asyncio.apply()  # Patch the running event loop

In [17]:
await main()

Scraping page 1...
Scraping drama: https://mydramalist.com/735043-life


/usr/local/lib/python3.11/dist-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Scraping drama: https://mydramalist.com/739603-sparkling-watermelon
Scraping drama: https://mydramalist.com/49231-move-to-heaven
Scraping drama: https://mydramalist.com/57173-hospital-playlist-2
Scraping drama: https://mydramalist.com/702267-weak-hero
Scraping drama: https://mydramalist.com/52939-can-this-person-be-translated
Scraping drama: https://mydramalist.com/54697-golden-hour
Scraping drama: https://mydramalist.com/54625-flower-of-evil
Scraping drama: https://mydramalist.com/13544-reply-1988
Scraping drama: https://mydramalist.com/36269-doctor-playbook
Scraping drama: https://mydramalist.com/25560-moving
Scraping drama: https://mydramalist.com/25172-my-ajusshi
Scraping drama: https://mydramalist.com/750099-time-walking-on-memory
Scraping drama: https://mydramalist.com/705857-umbrella
Scraping drama: https://mydramalist.com/35729-emergency-lands-of-love
Scraping drama: https://mydramalist.com/745581-the-glory-season-2
Scraping drama: https://mydramalist.com/58365-queen-cheorin
Sc